In [1]:
import $ivy.`org.apache.spark::spark-sql:3.1.2`
import $ivy.`org.apache.spark::spark-mllib:3.1.2`

import $ivy.$                                  

import $ivy.$                                    

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}


# Spark ML Production

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Spark ML Research")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/02/10 15:24:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/10 15:24:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/02/10 15:24:07 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@55ebb0d9
import spark.implicits._

## Загружаем данные

In [4]:
val basePath = "/home/vadim/Projects/otus/SparkML"

basePath: String = "/home/vadim/Projects/otus/SparkML"

In [5]:
val data = spark
        .read
        .option("header", "true")
        .option("inferSchema", "true")
        .csv(s"$basePath/data/BankChurners.csv")

data: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 21 more fields]

## Загружаем модель

In [6]:
import org.apache.spark.ml.PipelineModel

val model = PipelineModel.load(s"$basePath/pipelineModel")

import org.apache.spark.ml.PipelineModel


model: PipelineModel = pipeline_3df325fecde6

## Вычисляем

In [7]:
val prediction = model.transform(data)

prediction: DataFrame = [CLIENTNUM: int, Attrition_Flag: string ... 37 more fields]

In [8]:
prediction.show(10)

22/02/10 15:24:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------+-----------------------+--------------+-----------------------+---------------------+-----------------------------+----------------------------+--------------------+-----------------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|CLIENTNUM|   Attriti

## Проверяем результат

In [9]:
val tp = prediction.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 1)).count
val tn = prediction.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 0)).count
val fp = prediction.filter(($"Attrition_Flag" === "Existing Customer") and ($"prediction" === 1)).count
val fn = prediction.filter(($"Attrition_Flag" === "Attrited Customer") and ($"prediction" === 0)).count

println(s"Confusion Matrix:\n$tp\t$fp\n$fn\t\t$tn\n")

Confusion Matrix:
1101	1345
526		7155



tp: Long = 1101L
tn: Long = 7155L
fp: Long = 1345L
fn: Long = 526L

In [10]:
val accuracy = (tp + tn) / (tp + tn + fp + fn).toDouble
val precision = tp / (tp + fp).toDouble
val recall = tp / (tp + fn).toDouble

println(s"Accuracy = $accuracy")
println(s"Precision = $precision")
println(s"Recall = $recall\n")

Accuracy = 0.8152463710871927
Precision = 0.4501226492232216
Recall = 0.6767055931161647



accuracy: Double = 0.8152463710871927
precision: Double = 0.4501226492232216
recall: Double = 0.6767055931161647